In [1]:
import math
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns  
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
import sklearn.preprocessing
from sklearn.feature_selection import SelectKBest, f_regression, chi2, RFE
from env import username, host, password
import wrangle

In [2]:
zillow_df = wrangle.wrangle_zillow()

In [6]:
zillow_df.isnull().sum()

bed          0
bath         0
sqft         0
tax_value    0
yearbuilt    0
taxamount    0
fips         0
dtype: int64

In [10]:
zillow_df.describe().T

,count,mean,std,min,25%,50%,75%,max
bed,28021.0,3.314871,0.930545,0.00,3.00,3.00,4.00,12.00
bath,28021.0,2.319832,1.012109,0.00,2.00,2.00,3.00,11.00
sqft,28021.0,1933.928625,992.847731,240.00,1278.00,1674.00,2325.00,17245.00
tax_value,28021.0,533176.721709,703482.386246,1000.00,198613.00,382244.00,631825.00,23858374.00
yearbuilt,28021.0,1963.782520,23.098472,1878.00,1950.00,1961.00,1979.00,2016.00
taxamount,28021.0,6487.348151,8165.511201,51.26,2704.85,4734.55,7520.44,276797.83
fips,28021.0,6049.450162,21.229501,6037.00,6037.00,6037.00,6059.00,6111.00


In [13]:
# fips codes are unique county codes, there are 3 different counties here. Perhaps a driver?
zillow_df.fips.unique()

array([6037., 6059., 6111.])

In [14]:
zillow_df['tax_rate'] = zillow_df['taxamount']/zillow_df['tax_value']

In [24]:
zillow_df.describe().T

,count,mean,std,min,25%,50%,75%,max
bed,28021.0,3.314871,0.930545,0.000000,3.000000,3.0000,4.000000,1.200000e+01
bath,28021.0,2.319832,1.012109,0.000000,2.000000,2.0000,3.000000,1.100000e+01
sqft,28021.0,1933.928625,992.847731,240.000000,1278.000000,1674.0000,2325.000000,1.724500e+04
tax_value,28021.0,533176.721709,703482.386246,1000.000000,198613.000000,382244.0000,631825.000000,2.385837e+07
yearbuilt,28021.0,1963.782520,23.098472,1878.000000,1950.000000,1961.0000,1979.000000,2.016000e+03
taxamount,28021.0,6487.348151,8165.511201,51.260000,2704.850000,4734.5500,7520.440000,2.767978e+05
fips,28021.0,6049.450162,21.229501,6037.000000,6037.000000,6037.0000,6059.000000,6.111000e+03
tax_rate,28021.0,0.013337,0.007370,0.000094,0.011596,0.0123,0.013633,8.164900e-01


In [30]:
zillow_df[zillow_df['tax_rate'] == zillow_df['tax_rate'].min()]

,bed,bath,sqft,tax_value,yearbuilt,taxamount,fips,tax_rate
23220,7.0,4.5,3779.0,951513.0,1981.0,89.18,6111.0,0.000094


In [29]:
zillow_df

,bed,bath,sqft,tax_value,yearbuilt,taxamount,fips,tax_rate
0,3.0,2.0,1316.0,205123.0,1923.0,2627.48,6037.0,0.012809
1,3.0,2.0,1458.0,136104.0,1970.0,2319.90,6037.0,0.017045
2,2.0,1.0,1421.0,35606.0,1911.0,543.69,6037.0,0.015270
3,4.0,3.0,2541.0,880456.0,2003.0,9819.72,6059.0,0.011153
4,3.0,2.0,1491.0,107110.0,1955.0,1399.27,6037.0,0.013064
...,...,...,...,...,...,...,...,...
28119,6.0,2.0,2634.0,878190.0,1921.0,12766.88,6037.0,0.014538
28120,4.0,1.5,1187.0,465999.0,1959.0,5552.68,6059.0,0.011916
28121,5.0,3.0,3015.0,852813.0,2002.0,11662.88,6059.0,0.013676
28122,5.0,3.0,2992.0,1120000.0,1977.0,13663.03,6037.0,0.012199
